In [36]:
import os, sys
import multiprocessing as mp
import numpy as np

In [56]:
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten
from keras.models import Model
from keras.initializers import Constant

In [108]:
import pickle

In [120]:
cd newsgroups/

/home/joao/SCRIPTS/Drugs-Review-Dataset/CNN_TEXT/newsgroups


In [121]:
ls

20_newsgroup/  news20.tar.gz


In [122]:
cd 20_newsgroup/

/home/joao/SCRIPTS/Drugs-Review-Dataset/CNN_TEXT/newsgroups/20_newsgroup


In [123]:
ls

alt.atheism/               rec.autos/           sci.space/
comp.graphics/             rec.motorcycles/     soc.religion.christian/
comp.os.ms-windows.misc/   rec.sport.baseball/  talk.politics.guns/
comp.sys.ibm.pc.hardware/  rec.sport.hockey/    talk.politics.mideast/
comp.sys.mac.hardware/     sci.crypt/           talk.politics.misc/
comp.windows.x/            sci.electronics/     talk.religion.misc/
misc.forsale/              sci.med/


In [124]:
sorted(os.listdir())

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Read text files

In [12]:
def dir_operation(directory):
    texts=[]
    
    path = os.path.join(directory)
    if os.path.isdir(path):
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()

    return directory, texts[:]

In [13]:
with mp.Pool(mp.cpu_count()) as pool:
    result=pool.map(dir_operation,sorted(os.listdir()))

In [17]:
labels_index={name:i for i,name in enumerate(sorted(os.listdir()))}

In [18]:
labels_index

{'alt.atheism': 0,
 'comp.graphics': 1,
 'comp.os.ms-windows.misc': 2,
 'comp.sys.ibm.pc.hardware': 3,
 'comp.sys.mac.hardware': 4,
 'comp.windows.x': 5,
 'misc.forsale': 6,
 'rec.autos': 7,
 'rec.motorcycles': 8,
 'rec.sport.baseball': 9,
 'rec.sport.hockey': 10,
 'sci.crypt': 11,
 'sci.electronics': 12,
 'sci.med': 13,
 'sci.space': 14,
 'soc.religion.christian': 15,
 'talk.politics.guns': 16,
 'talk.politics.mideast': 17,
 'talk.politics.misc': 18,
 'talk.religion.misc': 19}

In [112]:
texts=[]
labels=[]
for tup in result:
    for t in tup[1]:
        texts.append(t)
        labels.append(labels_index[tup[0]])

In [110]:
pickle.dump(labels_index,open('labels_index.pkl','wb'))

In [116]:
pickle.dump(labels,open('labels.pkl','wb'))

In [109]:
pickle.dump(texts, open('texts.pkl','wb'))

In [ ]:
texts[:1]

In [28]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [33]:
MAX_NB_WORDS=20000

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
sequences[:1]

In [32]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 174074 unique tokens.


In [34]:
MAX_SEQUENCE_LENGTH=1000

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
data[:1]

In [38]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [39]:
VALIDATION_SPLIT=0.2

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [40]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [45]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/joao/.wget-hsts'. HSTS will be disabled.
--2020-08-06 14:52:14--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-08-06 14:52:14--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-08-06 14:52:15--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs

In [46]:
pwd

'/home/joao/SCRIPTS/Drugs-Review-Dataset/CNN_TEXT'

In [47]:
GLOVE_DIR='GLOVE'

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [49]:
len(word_index)

174074

In [102]:
EMBEDDING_DIM=100

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [67]:
len(word_index)

174074

In [103]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [77]:
MAX_SEQUENCE_LENGTH

1000

In [104]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

In [105]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [106]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=2,
          validation_data=(x_val, y_val))

Epoch 1/2
125/125 [==============================] - 46s 364ms/step - loss: 2.4401 - acc: 0.1971 - val_loss: 1.8763 - val_acc: 0.3451
Epoch 2/2
125/125 [==============================] - 52s 418ms/step - loss: 1.5725 - acc: 0.4486 - val_loss: 1.4648 - val_acc: 0.5064
